In [1]:
import pandas as pd
import numpy as np
import bokeh.plotting as bp
#bp.output_notebook()

In [9]:
#Se lee la señal anteriormente procesada
data_set = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_EURUSD-2017-07.csv", parse_dates=True,
                      infer_datetime_format=True, index_col=0)

In [21]:
%run ../code/EUtilities.py
%run ../code/PNLEstimatorWrapper
EU = EUtilities()

In [11]:
data_set[:10]

,bid,ask
date,,
2017-07-03 00:00:10,1.14183,1.14211
2017-07-03 00:00:20,1.14163,1.14211
2017-07-03 00:00:30,1.14164,1.14206
2017-07-03 00:00:50,1.14164,1.14206
2017-07-03 00:01:00,1.14164,1.14208
2017-07-03 00:01:10,1.14164,1.14206
2017-07-03 00:01:20,1.14163,1.14206
2017-07-03 00:01:30,1.14156,1.14206
2017-07-03 00:01:40,1.14164,1.14208


In [12]:
window = 3
X,y_reg,bt= EU.build_dataset(data_set,window=window, PNL=True,binary_target=True,delete_constant_values=True)

In [15]:
X.head()

,0,1,2,PNL_0,PNL_1
date,,,,,
2017-07-03 00:01:10,1.14164,1.14164,1.14164,-0.00042,-0.00043
2017-07-03 00:01:20,1.14164,1.14164,1.14163,-0.00043,-0.00050
2017-07-03 00:01:30,1.14164,1.14163,1.14156,-0.00052,-0.00042
2017-07-03 00:02:00,1.14164,1.14164,1.14164,-0.00042,-0.00051
2017-07-03 00:02:10,1.14164,1.14164,1.14156,-0.00052,-0.00042


In [22]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_GNB)
#df_GNB.to_csv('../resultados/TrueFX/GBP-USD/GaussianNB/pnl_GNB_wsize3_2018-01.csv')

--------- GaussianNB ----------------
average PNL :  -0.239068333333
average buy:  554.722222222
average sell:  6060.27777778
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2017-07-07 -0.23658     0.0  7013.0           0.0         7013.0
2017-07-10 -0.24576     0.0  6535.0           0.0         6535.0
2017-07-11 -0.20856     0.0  6687.0           0.0         6687.0
2017-07-12 -0.23936     0.0  6995.0           0.0         6995.0
2017-07-13 -0.22130  3035.0  3861.0        1685.0         2981.0
2017-07-14 -0.22834  2344.0  4376.0        1714.0         3450.0
2017-07-17 -0.31109     0.0  6631.0           0.0         6631.0
2017-07-18 -0.22088     0.0  7201.0           0.0         7201.0
2017-07-19 -0.21661     0.0  6745.0           0.0         6745.0
2017-07-20 -0.22630     0.0  7096.0           0.0         7096.0
2017-07-21 -0.23226     0.0  7054.0           0.0         7054.0
2017-07-24 -0.32235     0.0  6832.0           0.0       

In [23]:
from sklearn.neighbors import KNeighborsClassifier
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(), PNL_column=['PNL_1','PNL_0'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
print('--------- KNeighborsClassifier ------------')
print('average PNL : ', np.mean(result_KNC[:,0]))
print('average buy: ', np.mean(result_KNC[:,1]))
print('average sell: ', np.mean(result_KNC[:,2]))
print('-------------------------------------')
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_KNC)

--------- KNeighborsClassifier ------------
average PNL :  -0.240473333333
average buy:  2800.72222222
average sell:  3814.27777778
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2017-07-07 -0.22801  3324.0  3689.0          21.0           18.0
2017-07-10 -0.24287  3034.0  3501.0          19.0           20.0
2017-07-11 -0.21173  2511.0  4176.0          25.0         1294.0
2017-07-12 -0.24316  3815.0  3180.0         401.0          340.0
2017-07-13 -0.21132  3357.0  3539.0          20.0           48.0
2017-07-14 -0.22960  3254.0  3466.0          20.0           17.0
2017-07-17 -0.31092  3134.0  3497.0          20.0           28.0
2017-07-18 -0.23275   501.0  6700.0           9.0         6145.0
2017-07-19 -0.21063  3766.0  2979.0         173.0           30.0
2017-07-20 -0.24639  3566.0  3530.0        1327.0         2043.0
2017-07-21 -0.23485  1566.0  5488.0          18.0         2100.0
2017-07-24 -0.31746  3116.0  3716.0          15.0 

In [24]:
from sklearn.tree import DecisionTreeClassifier

est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(), PNL_column=['PNL_1','PNL_0'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,4,1,1))
result_DTC, predict_DTC = EU.redim(result_DTC)
print('------ DecisionTreeClassifier -------')
print('average PNL: ', np.mean(result_DTC[:,0]))
print('average buy: ', np.mean(result_DTC[:,1]))
print('average sell: ', np.mean(result_DTC[:,2]))
print('-------------------------------------')
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_DTC)

------ DecisionTreeClassifier -------
average PNL:  -0.240777222222
average buy:  2868.94444444
average sell:  3746.05555556
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2017-07-07 -0.22663  3337.0  3676.0          18.0           31.0
2017-07-10 -0.24264  2983.0  3552.0          14.0           17.0
2017-07-11 -0.21304  3748.0  2939.0        1141.0           41.0
2017-07-12 -0.23276  4224.0  2771.0         792.0           29.0
2017-07-13 -0.21753  3337.0  3559.0          31.0           22.0
2017-07-14 -0.23230  3098.0  3622.0          14.0           13.0
2017-07-17 -0.30280  3118.0  3513.0          12.0           14.0
2017-07-18 -0.23317   480.0  6721.0           8.0         6166.0
2017-07-19 -0.22331  3679.0  3066.0          94.0           33.0
2017-07-20 -0.23875  2982.0  4114.0         208.0         2039.0
2017-07-21 -0.22687  1962.0  5092.0           9.0         2010.0
2017-07-24 -0.32073  3338.0  3494.0          27.0        

In [25]:
from sklearn.ensemble import RandomForestClassifier

est_RFC = PNLEstimatorWrapper(RandomForestClassifier(n_estimators=15, n_jobs=3), PNL_column=['PNL_1','PNL_0'])
result_RFC = EU.step_validation(est_RFC, X, y, EU.v_split(X,4,1,1))
result_RFC, predict_RFC = EU.redim(result_RFC)
print('------- RandomForestClassifier ----------')
print('PNL promedio: ', np.mean(result_RFC[:,0]))
print('buy promedio: ', np.mean(result_RFC[:,1]))
print('sell promedio: ', np.mean(result_RFC[:,2]))
print('-------------------------------------')
df_RFC = pd.DataFrame(result_RFC, columns=list(['PNL','buys','sells','longest_buys','longest_sells']),index=dates)
print(df_RFC) 

------- RandomForestClassifier ----------
PNL promedio:  -0.240703333333
buy promedio:  2779.77777778
sell promedio:  3835.22222222
-------------------------------------
                PNL    buys   sells  longest_buys  longest_sells
2017-07-07 -0.22315  3268.0  3745.0          14.0           30.0
2017-07-10 -0.24377  2925.0  3610.0          12.0           22.0
2017-07-11 -0.21369  2509.0  4178.0          14.0         1291.0
2017-07-12 -0.22787  3875.0  3120.0         793.0          319.0
2017-07-13 -0.21382  3568.0  3328.0          23.0           15.0
2017-07-14 -0.22897  3177.0  3543.0          11.0           14.0
2017-07-17 -0.30113  3175.0  3456.0          14.0           14.0
2017-07-18 -0.23149   500.0  6701.0           7.0         6162.0
2017-07-19 -0.21457  3471.0  3274.0          92.0           55.0
2017-07-20 -0.24194  3571.0  3525.0        1252.0         2044.0
2017-07-21 -0.23089  1591.0  5463.0          16.0         2031.0
2017-07-24 -0.32317  3232.0  3600.0          18.0 